# 4. Stitch outputs together
- Once you have fitted the weak, medium and dense diads, stitch them all together, and apply the Ne correction model

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import DiadFit as pf
pf.__version__

'0.0.63'

In [2]:
import os
DayFolder=os.getcwd()
meta_path=DayFolder +  '\MetaData'
spectra_path=DayFolder + '\Spectra'
Trupower=True

## Load in specra, and combine

In [3]:
df2=pf.merge_fit_files()

## First, get the metadata for this day

In [4]:
# Put the common string you can in all your Ne lines here
ID_str='_'
file_ext_meta='txt' # Will only take files of this type 
diad_meta=pf.get_files(path=meta_path,
file_ext='txt', exclude_str=['Ne', 'FG', 'Cap','window'],
 sort=True)
diad_meta[0]

'K21-59-FI#1-IMG (1).txt'

## Now get all the important information out of the metadata files

In [5]:
meta=pf.stitch_metadata_in_loop_witec(path=meta_path, 
Allfiles=diad_meta, 
trupower=Trupower)
print(meta['filename'])

100%|██████████| 42/42 [00:00<00:00, 45.99it/s]

Done
0            K21-59-FI#1.txt
1            K21-59-FI#2.txt
2            K21-59-FI#3.txt
3       K21-61-FI#1-rep1.txt
4       K21-61-FI#1-rep2.txt
5            K21-61-FI#2.txt
6            K21-61-FI#3.txt
7     K21-62-FI#1-nodiad.txt
8     K21-62-FI#2-nodiad.txt
9            K21-63-FI#1.txt
10           K21-64-FI#1.txt
11           K21-64-FI#2.txt
12           K21-64-FI#3.txt
13           K21-64-FI#4.txt
14           K21-64-FI#5.txt
15           K21-64-FI#6.txt
16           K21-64-FI#7.txt
17           K21-64-FI#8.txt
Name: filename, dtype: object


## Then get a simple file name you can stitch with the spectra

In [6]:
file_m=pf.extracting_filenames_generic(names=meta['filename'],
   file_type='.txt')
# Checks they are all unique
file_m[0]

good job, no duplicate file names


'K21-59-FI#1'

## Now get filename from spectra into same form as metadata

In [7]:
# Remove these to get the pure file name
file_s=pf.extracting_filenames_generic(
    prefix=prefix, str_prefix=str_prefix,
    names=df2['filename'].reset_index(drop=True),
   file_type='.txt')

# Remove the cosmic rays, as doesnt have it in the metatdata
file_s = np.char.replace(file_s.astype(str), "_CRR_DiadFit", "")

good job, no duplicate file names


In [8]:
# Combining them together
meta['name_for_matching']=file_m
df2['name_for_matching']=file_s
df2['Name_for_Secondary_Phases']=file_s
df_combo=pd.merge(df2, meta, on='name_for_matching').reset_index(drop=True)
df_combo.head()

,filename_x,Splitting,Split_σ,Split_err_abs,Diad1_Combofit_Cent,Diad1_cent_err,Diad1_Combofit_Height,Diad1_Voigt_Cent,Diad1_Voigt_Area,Diad1_Voigt_Sigma,...,Month,Day,power (mW),Int_time (s),accumulations,Mag (X),duration,24hr_time,sec since midnight,Spectral Center
0,K21-59-FI#1_CRR_DiadFit,103.057930,0.004029,0.005424,1286.035372,0.003584,591.792421,1286.035422,605.447362,0.434264,...,October,28.0,12.078,45.0,5.0,50.0,"['0h', '3m', '47s']",12:09:55 PM\n,43795.0,1325.004
1,K21-59-FI#2,103.011311,0.011688,0.016082,1286.157455,0.009950,227.655256,1286.157505,198.574397,0.409721,...,October,28.0,12.068,45.0,5.0,50.0,"['0h', '3m', '47s']",12:19:53 PM\n,44393.0,1325.004
2,K21-59-FI#3,103.046227,0.003908,0.005329,1286.074215,0.003398,570.233153,1286.074265,580.899302,0.422487,...,October,28.0,12.064,45.0,5.0,50.0,"['0h', '3m', '47s']",12:32:26 PM\n,45146.0,1325.004
3,K21-61-FI#1-rep1_CRR_DiadFit,103.312261,0.031987,0.042104,1285.431180,0.029323,66.226323,1285.431180,80.426681,0.570436,...,October,28.0,12.046,45.0,5.0,50.0,"['0h', '3m', '47s']",12:45:44 PM\n,45944.0,1325.004
4,K21-61-FI#1-rep2_CRR_DiadFit,103.305046,0.003063,0.004135,1285.435753,0.002712,607.647353,1285.435803,895.504629,0.568594,...,October,28.0,12.030,45.0,5.0,50.0,"['0h', '3m', '46s']",12:50:47 PM\n,46247.0,1325.004


## If you have secondary phases, now is the time to merge those in

In [9]:
df_combo_sec_phase=pf.merge_in_carb_SO2(
df_combo=df_combo, file1_name='Carb_Peak_fits.xlsx', file2_name='SO2_Peak_fits.xlsx')
df_combo_sec_phase.to_excel('FI_fitting_w_sec_phases.xlsx', index=False)

Made a df!
good job, no duplicate file names


In [10]:
df_combo_sec_phase.head()

,filename_x,Splitting,Split_σ,Split_err_abs,Diad1_Combofit_Cent,Diad1_cent_err,Diad1_Combofit_Height,Diad1_Voigt_Cent,Diad1_Voigt_Area,Diad1_Voigt_Sigma,...,filename,Peak_Cent_SO2,Peak_Area_SO2,Peak_Height_SO2,Model_name,Peak_Cent_Carb,Peak_Area_Carb,Peak_Height_Carb,Carb_Diad_Ratio,SO2_Diad_Ratio
0,K21-59-FI#1_CRR_DiadFit,103.057930,0.004029,0.005424,1286.035372,0.003584,591.792421,1286.035422,605.447362,0.434264,...,K21-59-FI#1.txt,1151.271035,49.794305,26.997104,Spline,NaN,NaN,NaN,NaN,0.032879
1,K21-59-FI#2,103.011311,0.011688,0.016082,1286.157455,0.009950,227.655256,1286.157505,198.574397,0.409721,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,K21-59-FI#3,103.046227,0.003908,0.005329,1286.074215,0.003398,570.233153,1286.074265,580.899302,0.422487,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,K21-61-FI#1-rep1_CRR_DiadFit,103.312261,0.031987,0.042104,1285.431180,0.029323,66.226323,1285.431180,80.426681,0.570436,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,K21-61-FI#1-rep2_CRR_DiadFit,103.305046,0.003063,0.004135,1285.435753,0.002712,607.647353,1285.435803,895.504629,0.568594,...,K21-61-FI#1-rep2.txt,NaN,NaN,NaN,NaN,1090.919723,900.825267,43.734455,0.38602,NaN
